<a href="https://colab.research.google.com/github/pasumarthi/EIP4/blob/master/ImageAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# function to add label smoothing
def add_label_noise(y,noise_level=0.2,selected_cols=[]):
    def add_noise(x):
        temp = x*(1-noise_level)
        res = (np.asarray(([noise_level]*x.shape[1]))/x.shape[1]).reshape(1,-1)
        temp=temp+res
        return temp
    if len(selected_cols)==0:
        temp=add_noise(y)
    else:
        temp=y.copy()
        for c in selected_cols:
            temp[temp[c]==1] = add_noise(temp[temp[c]==1])  
    return temp

In [0]:
from imgaug import augmenters as iaa

aug1 = iaa.CoarseDropout(p=0.10, size_percent=0.05)

def additional_augmenation(image):
    image = aug1.augment_image(image)

    return image

In [0]:
train_gen = PersonDataGenerator(train_df, 
                                batch_size=32, 
                                augmentation=ImageDataGenerator( 
                                    # set input mean to 0 over the dataset 
                                    featurewise_center=False, 
                                    # set each sample mean to 0 
                                    samplewise_center=False, 
                                    # divide inputs by std of dataset 
                                    featurewise_std_normalization=False, 
                                    # divide each input by its std 
                                    samplewise_std_normalization=False, 
                                    # apply ZCA whitening 
                                    zca_whitening=False, 
                                    # epsilon for ZCA whitening 
                                    zca_epsilon=1e-06, 
                                    # randomly rotate images in the range (deg 0 to 180) 
                                    rotation_range=0, 
                                    # randomly shift images horizontally 
                                    width_shift_range=0.1, 
                                    # randomly shift images vertically 
                                    height_shift_range=0.1, 
                                    # set range for random shear 
                                    shear_range=0.15, 
                                    # set range for random zoom 
                                    zoom_range=0., 
                                    # set range for random channel shifts 
                                    channel_shift_range=0., 
                                    # set mode for filling points outside the input boundaries 
                                    fill_mode='nearest', 
                                    # value used for fill_mode = "constant" 
                                    cval=0., 
                                    # randomly flip images 
                                    horizontal_flip=True, 
                                    # randomly flip images 
                                    vertical_flip=False, 
                                    # set rescaling factor (applied before any other transformation) 
                                    rescale=None, 
                                    # set function that will be applied on each input 
                                    preprocessing_function=additional_augmenation, 
                                    # image data format, either "channels_first" or "channels_last" 
                                    data_format=None, 
                                    # fraction of images reserved for validation (strictly between 0 and 1) 
                                    validation_split=0.0))

valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False)


In [0]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser